In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('04-19-labels.csv')
# Eliminamos las columnas de los timestamps
df = df.dropna(axis=1, how='all')

Vamos a agrupar por `User`, `Replay` y `Group` y a crear una nueva columna por etiqueta para poder manipular más fácilmente los datos.

In [2]:
df_pivoted = df.pivot_table(index=["User", "Replay","Group"], columns="Competence", values="Tag", fill_value=None).reset_index()

#df_pivoted.head()

Hay constructos para los que utilizamos más de una etiqueta, por lo que antes de hacer la media final para obtener el valor para el razonamiento espacial, vamos a hacer la media para cada constructo.

In [3]:
# Agrupamos las columnas por constructos
constructs = {
    "mental_rotation": ["rotation"],
    "spatial_orientation": ["cam_perspective", "snapshots"],
    "spatial_structuring": ["prim_shapes", "scale_shapes", "locate_shapes"],
    "spatial_visualization": ["action_seq"]
}
### Función auxiliar que me ha hecho chatGPT ##
# Calcular manualmente las medias para cada constructo, ignorando los ceros
def calculate_mean_without_zeros(df, columns):
    # Calcula la media de las columnas especificadas en df, ignorando los ceros
    means = []
    for index, row in df.iterrows():
        values = [row[col] for col in columns if row[col] != 0]  # Ignorar ceros
        mean = sum(values) / len(values) if values else 0  # Evitar división por cero
        means.append(mean)
    return means

# Creamos una nueva columna por cada constructo
for construct, columns in constructs.items():
    #df_pivoted[construct] = df_pivoted[columns].mean(axis=1,skipna=True)
    df_pivoted[construct] = calculate_mean_without_zeros(df_pivoted, columns)

df_pivoted["spatial_reasoning"] = calculate_mean_without_zeros(df_pivoted,constructs.keys())
#df_pivoted["spatial_reasoning_alt"] = calculate_mean_without_zeros(df_pivoted, np.concatenate(list(constructs.values())).tolist())
df_pivoted.head()

Competence,User,Replay,Group,action_seq,cam_perspective,locate_shapes,prim_shapes,rotation,scale_shapes,snapshots,mental_rotation,spatial_orientation,spatial_structuring,spatial_visualization,spatial_reasoning
0,00f86c5fc28e8d326cdae17ea97c7387,45-Degree Rotations~1,e6af7d42084352a39449e6d0a09b18cd,3,3,4,0,3,0,4,3.0,3.5,4.000000,3.0,3.375000
1,00f86c5fc28e8d326cdae17ea97c7387,Angled Silhouette~1,e6af7d42084352a39449e6d0a09b18cd,2,2,3,4,3,4,2,3.0,2.0,3.666667,2.0,2.666667
2,00f86c5fc28e8d326cdae17ea97c7387,Bird Fez~2,e6af7d42084352a39449e6d0a09b18cd,3,3,3,4,4,4,4,4.0,3.5,3.666667,3.0,3.541667
3,00f86c5fc28e8d326cdae17ea97c7387,Pi Henge~1,e6af7d42084352a39449e6d0a09b18cd,2,2,4,3,4,4,4,4.0,3.0,3.666667,2.0,3.166667
4,00f86c5fc28e8d326cdae17ea97c7387,Pyramids are Strange~1,e6af7d42084352a39449e6d0a09b18cd,2,3,4,0,2,4,1,2.0,2.0,4.000000,2.0,2.500000


In [4]:
'''
import matplotlib.pyplot as plt

# Para poder ver la diferencia entre una cuenta y la otra vamos a ordenar por una de ellas
df_pivoted_sorted = df_pivoted.sort_values(by="spatial_reasoning")

# Asumiendo que ya tienes las columnas 'spatial_reasoning' y 'spatial_reasoning_alt' en tu DataFrame
# Vamos a crear un gráfico para visualizar la diferencia entre estas dos columnas

plt.figure(figsize=(10, 6))

indices_sorted = range(len(df_pivoted_sorted))

# Plotear 'spatial_reasoning' después de ordenar
plt.plot(indices_sorted, df_pivoted_sorted['spatial_reasoning'], label='Spatial Reasoning', marker='o')

# Plotear 'spatial_reasoning_alt' usando el mismo orden
plt.plot(indices_sorted, df_pivoted_sorted['spatial_reasoning_alt'], label='Spatial Reasoning Alt', marker='x')



plt.title('Comparación de Spatial Reasoning vs Spatial Reasoning Alt')
plt.xlabel('Índice')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.show()
'''

'\nimport matplotlib.pyplot as plt\n\n# Para poder ver la diferencia entre una cuenta y la otra vamos a ordenar por una de ellas\ndf_pivoted_sorted = df_pivoted.sort_values(by="spatial_reasoning")\n\n# Asumiendo que ya tienes las columnas \'spatial_reasoning\' y \'spatial_reasoning_alt\' en tu DataFrame\n# Vamos a crear un gráfico para visualizar la diferencia entre estas dos columnas\n\nplt.figure(figsize=(10, 6))\n\nindices_sorted = range(len(df_pivoted_sorted))\n\n# Plotear \'spatial_reasoning\' después de ordenar\nplt.plot(indices_sorted, df_pivoted_sorted[\'spatial_reasoning\'], label=\'Spatial Reasoning\', marker=\'o\')\n\n# Plotear \'spatial_reasoning_alt\' usando el mismo orden\nplt.plot(indices_sorted, df_pivoted_sorted[\'spatial_reasoning_alt\'], label=\'Spatial Reasoning Alt\', marker=\'x\')\n\n\n\nplt.title(\'Comparación de Spatial Reasoning vs Spatial Reasoning Alt\')\nplt.xlabel(\'Índice\')\nplt.ylabel(\'Valor\')\nplt.legend()\nplt.grid(True)\nplt.show()\n'

In [5]:
# Vamos a ver los percentiles
df_pivoted[["spatial_reasoning"]].describe()

Competence,spatial_reasoning
count,113.000000
mean,3.073378
std,0.635910
min,1.500000
25%,2.625000
50%,3.222222
75%,3.500000
max,4.000000


In [6]:
df_pivoted[["User","Replay","spatial_reasoning"]].to_csv("04-19-labels-constructs.csv", index=False)
